<a href="https://colab.research.google.com/github/Rcgil30/Lab2ED2/blob/main/Datos_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import folium
import numpy as np


flights_url ='https://raw.githubusercontent.com/Rcgil30/Lab2ED2/main/data/aerocivil.csv?token=GHSAT0AAAAAABYNZVOSYKAQXCAMFQQXRX2SYZOGQTA'
flights = pd.read_csv(flights_url)

cities_url ='https://raw.githubusercontent.com/Rcgil30/Lab2ED2/main/data/cities_cord.csv?token=GHSAT0AAAAAABYNZVOT5XKRGNAMUEOHM3VOYZOGQ4A'
cities = pd.read_csv(cities_url)

cities = cities[['city','lat','lng','country','admin_name', 'capital']]
flights.columns = [c.replace(' ', '_') for c in flights.columns]
flights = flights[['Origen','Ciudad_Origen','Destino','Ciudad_Destino','Tráfico_(N/I)','Tipo_Vuelo','Pasajeros']]
flights = flights[(flights['Tráfico_(N/I)'] == "N") & (flights['Tipo_Vuelo'] == "R") & (flights['Pasajeros'] >= 680)] 

In [3]:
cities  = cities[(cities['country'] == "Colombia")]
cities['city'] = cities['city'].str.upper()
cities['admin_name'] = cities['admin_name'].str.upper()

main_flights = flights[(((flights['Ciudad_Destino']).isin(cities.city)) & ((flights['Ciudad_Origen']).isin(cities.city)))]
main_flights = main_flights.drop_duplicates(subset=['Ciudad_Origen','Ciudad_Destino'],keep= "first")

cords = {'lat_st': [],'lng_st': [],'lat_end': [],'lng_end': []}

def add_cords(df, par, lat, lng):
  for x in df[par]:
    loc = cities.loc[cities['city'].eq(x)]
    cords[lat].append(float(loc.lat))
    cords[lng].append(float(loc.lng))

add_cords(main_flights, "Ciudad_Origen","lat_st","lng_st")
add_cords(main_flights, "Ciudad_Destino","lat_end","lng_end")
cordsdf = pd.DataFrame(cords)

main_flights['lat_st'] = cordsdf['lat_st'].values
main_flights['lng_st'] = cordsdf['lng_st'].values
main_flights['lat_end'] = cordsdf['lat_end'].values
main_flights['lng_end'] = cordsdf['lng_end'].values
main_flights
 

,Origen,Ciudad_Origen,Destino,Ciudad_Destino,Tráfico_(N/I),Tipo_Vuelo,Pasajeros,lat_st,lng_st,lat_end,lng_end
2308,ADZ,SAN ANDRES,BOG,BOGOTA,N,R,16500.0,12.5847,-81.7006,4.6126,-74.0705
2309,ADZ,SAN ANDRES,CLO,CALI,N,R,6694.0,12.5847,-81.7006,3.4400,-76.5197
2310,ADZ,SAN ANDRES,CTG,CARTAGENA,N,R,7346.0,12.5847,-81.7006,10.4236,-75.5253
2311,ADZ,SAN ANDRES,MDE,RIONEGRO,N,R,7759.0,12.5847,-81.7006,6.1533,-75.3742
2312,AXM,ARMENIA,BOG,BOGOTA,N,R,3527.0,4.5389,-75.6725,4.6126,-74.0705
...,...,...,...,...,...,...,...,...,...,...,...
5658,NVA,NEIVA,CTG,CARTAGENA,N,R,2599.0,2.9275,-75.2875,10.4236,-75.5253
5660,PEI,PEREIRA,ADZ,SAN ANDRES,N,R,5220.0,4.8143,-75.6946,12.5847,-81.7006
5722,SMR,SANTA MARTA,CUC,CUCUTA,N,R,1008.0,11.2361,-74.2017,7.9075,-72.5047
5751,VVC,VILLAVICENCIO,CTG,CARTAGENA,N,R,2306.0,4.1425,-73.6294,10.4236,-75.5253


In [451]:

def haversine_distance(lat1, lon1, lat2, lon2, r=6371):
    coordinates = lat1, lon1, lat2, lon2
    phi1, lambda1, phi2, lambda2 = [
        radians(c) for c in coordinates
    ]  
    a = (np.square(sin((phi2-phi1)/2)) + cos(phi1) * cos(phi2) * 
         np.square(sin((lambda2-lambda1)/2)))
    d = 2*r*asin(np.sqrt(a))
    return d

main_flights['distance'] = main_flights.apply(lambda row : haversine_distance(row['lat_st'],
                     row['lng_st'], row['lat_end'], row['lng_end']), axis = 1)

main_flights
   

,Origen,Ciudad_Origen,Destino,Ciudad_Destino,Tráfico_(N/I),Tipo_Vuelo,Pasajeros,lat_st,lng_st,lat_end,lng_end,distance
2308,ADZ,SAN ANDRES,BOG,BOGOTA,N,R,16500.0,12.5847,-81.7006,4.6126,-74.0705,1219.962502
2309,ADZ,SAN ANDRES,CLO,CALI,N,R,6694.0,12.5847,-81.7006,3.4400,-76.5197,1165.618728
2310,ADZ,SAN ANDRES,CTG,CARTAGENA,N,R,7346.0,12.5847,-81.7006,10.4236,-75.5253,714.435523
2311,ADZ,SAN ANDRES,MDE,RIONEGRO,N,R,7759.0,12.5847,-81.7006,6.1533,-75.3742,996.298160
2312,AXM,ARMENIA,BOG,BOGOTA,N,R,3527.0,4.5389,-75.6725,4.6126,-74.0705,177.755471
...,...,...,...,...,...,...,...,...,...,...,...,...
5658,NVA,NEIVA,CTG,CARTAGENA,N,R,2599.0,2.9275,-75.2875,10.4236,-75.5253,833.941319
5660,PEI,PEREIRA,ADZ,SAN ANDRES,N,R,5220.0,4.8143,-75.6946,12.5847,-81.7006,1087.025434
5722,SMR,SANTA MARTA,CUC,CUCUTA,N,R,1008.0,11.2361,-74.2017,7.9075,-72.5047,414.249968
5751,VVC,VILLAVICENCIO,CTG,CARTAGENA,N,R,2306.0,4.1425,-73.6294,10.4236,-75.5253,729.028118
